## 11.1 파이썬 문화에서의 인터페이스와 프로토콜
- 인터페이스 : 객체의 공개 메서드의 일부
- 프로토콜 : 역할을 완수하기 위한 메서드의 집합으로서의 인터페이스, 상속과 무관, 객체의 다형성 구현

## 11.2 파이썬은 시퀀스를 찾아낸다

In [1]:
# abc.Sequence 를 상속하지 않는 객체, __getitem__ 만 구현
class Foo:
    def __getitem__(self, pos):
        return range(0,30,10)[pos]

In [3]:
# __iter__() 메서드가 없지만 대체 수단인 __getitem__() 메서드가 구현되어 있으므로 반복, in 연산자 
f = Foo()
20 in f

True

In [1]:
# 1장에서 구현한 FrenchDeck 클래스도 abc.Sequence를 상속하지 않지만, 시퀀스 프로토콜의 __getitem__()과 __len__() 메서드를 구현한다.

import collections

Card = collections.namedtuple('Card', ['rank','suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2,11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                      for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, pos):
        return self._cards[pos]

## 11.3 런타임에 프로토콜을 구현하는 monkey patching
- FrenchDeck 클래스에 random.shuffle() 함수 사용 
- 다음과 같이 입력하면 예외가 발생하는데, 이는 FrenchDeck 객체가 할당을 지원하지 않기 때문이다.


In [2]:
from random import shuffle
deck = FrenchDeck()
shuffle(deck)

TypeError: 'FrenchDeck' object does not support item assignment

In [3]:
# 동적 언어의 특성을 활용하여 런타임에서 메서드 추가
# monkey patching

def set_card(deck, pos, card):
    deck._cards[pos] = card
    
FrenchDeck.__setitem__ = set_card

shuffle(deck)
deck[:5]

[Card(rank='3', suit='clubs'),
 Card(rank='Q', suit='spades'),
 Card(rank='8', suit='diamonds'),
 Card(rank='5', suit='spades'),
 Card(rank='10', suit='clubs')]

## 11.4 알렉스 마르텔리의 물세 
- abc 가 파이썬에 도움을 주는 이유 
- 덕 타이핑 : 객체의 실제 자료형 무시 
- 객체 용도에 맞는 메서드 이름, 시그니처, 의미를 구현하도록 보장 
- isinstance() 함수 사용을 회피 --> 상속을 할 수 없게 만듬
- 관측할 수 있는 특징에 주안점을 둔다 
- 같은 메서드 명을 공유할때 객체가 대등하다는 것을 보장할 수 없다. 메서드가 같으리라는 보장도 없다 
- goose typing 으로 덕타이핑을 보완
- cls 가 추상베이스인 경우 (cls 의 메타클래스가 abc.ABCMeta 인 경우) --> isinstance(obj, cls) 를 사용
- register() : 특정 클래스를 ABC 의 가상 서브클래스로 선언

In [4]:
# len 만을 구현함으로 abc.Size 서브클래스로 인식하게됨
class Struggle:
    def __len__(self): return 23

from collections import abc
isinstance(Struggle(), abc.Sized)

True

In [8]:
#  isinstance() 를 많이 사용하는 것은 코드 악취
#  if/else 블록안에서 isinstance를 쓰는 것은 좋지 않다 --> 다형성을 사용해야한다 

## 11.5 ABC 상속하기 

In [17]:
# FrenchDeck2를 colletions.MutableSequence의 서브클래스로 선언한다.
import collections

Card = collections.namedtuple('Card', ['rank','suit'])

class FrenchDeck2(collections.abc.MutableSequence):
    ranks = [str(n) for n in range(2,11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                      for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]
    
    def __setitem__(self, position, value):
        self._cards[position] = value
        
    # MutableSequence를 상속하므로 이 클래스의 추상 메서드인 
    # __delitem__() 도 구현해야함
    def __delitem__(self, position):
        del self._cards[position]
        
    # insert() 또한 추상 메서드, 구현하지 않을 경우 에러발생
    # def insert(self, position, value):
    #     self._cards.insert(position, value)

In [18]:
# 파이썬은 모듈을 로딩하거나 컴파일할 때가 아니라 실제로 객체를 생성할 때 추상 메서드의 구현 여부를 확인한다.
frenchdeck2 = FrenchDeck2()


TypeError: Can't instantiate abstract class FrenchDeck2 with abstract method insert

## 11.6 표준 라이브러리 ABC 
### 11.6.1 collections.abc 의 ABC
- 대부분의 ABC 는 collections.abc 모듈에 정의되어 있음. 
- iterable, Container, Sized : 모든 컬렉션은 이 abc 상속 혹은, 호환 프로토콜을 구현해야함
- Sequence, Mapping, Set : 주요 불변 컬렉션
- MappingView : items, keys, values 메서드에서 반환 되는 객체는 ItemsView, KeysView, ValuesView 를 상속 
- Callable, Hashable : 객체 호출 혹은 해싱이 가능한지 isinstance() 함수와 함께 사용
- Iterator : Iterable을 상속

### 11.6.2 ABC 숫자탑 
- 상위 Number --> Complex --> Real --> Rational --> Integral 
- 정수형인지 검사 isinstance(x, numbers.Integral)
- decimal.Decimal 은 numbers.Real 의 가상 서브클래스로 등록되어 있지 않음 --> decimal 과 float 은 다르다

In [20]:
import numbers

a = 1+ 2j
b = 3+ 3j
print(a*b, isinstance(a, numbers.Complex))

c = 3.0
print(c, isinstance(c, numbers.Integral), isinstance(c, numbers.Complex))

(-3+9j) True
3.0 False True


## 11.7 ABC의 정의와 사용 
- 프레임워크를 확장해야하는 상황을 가정 
- 광고관리 프레임워크 
- 무반복, 무작위 선발 클래스를 지원 --> 갖춰야할 특성을 알려주기위해 ABC를 정의
- 유한 집합에서 무작위 항목 선택 

1. Tobola ABC
- 추상 메서드 
    - load() : 항목을 컨테이너에 넣기
    - pick() : 항목을 무작위로 꺼내기
- 구상 메서드
    - loaded() : 항목이 1개 이상 있으면 True 를 반환
    - inspect() : 컨테이너 내부 항목으로부터 정렬된 튜플 반환 

In [5]:
# tombola.py

import abc

class Tombola(abc.ABC): # ABC 정의하기 위해 상속
    
    @abc.abstractmethod
    def load(self, iterable):
        """iterable의 항목들을 추가한다."""
        
    @abc.abstractmethod
    def pick(self):
        """무작위로 항목을 하나 제거하고 반환한다.
        객체가 비어 있을 때 이 메서드를 실행하면 'LookupError'가 발생한다.
        """
        
    def loaded(self): # ABC 에도 구상 메서드가 들어갈 수 있다
        """최소 한 개의 항목이 있으면 True, 아님 False 반환"""
        # ABC의 구상 메서드는 반드시 ABC에 정의된 인터페이스, 즉
        # ABC의 다른 구상 메서드나 추상 메서드, 혹은 프로퍼티만 사용해야 한다.
        return bool(self.inspect())
    
    def inspect(self):
        """현재 안에 있는 항목들로 구성된 정렬된 튜플 반환"""
        items = []
        while True:
            try:
                # pick() 을 계속 호출해서 Tombola 객체를 비움
                items.append(self.pick())
            except LookupError:
                break
        self.load(items) # load 메서드를 호출해서 다시 넣는다
        return tuple(sorted(items))

- inspect 는 비싼 연산을 수행  --> 추후 개선 

In [26]:
class Fake(Tombola):
    def pick(self):
        return 13

Fake

__main__.Fake

In [27]:
# load()를 구현하지 않았으므로 객체 생성 시 TypeError 발생 ()
f = Fake()

TypeError: Can't instantiate abstract class Fake with abstract method load

In [3]:
# bingo.py

import random
# from tombola import Tombola

class BingoCage(Tombola):
    def __init__(self, items):
        self._randomizer = random.SystemRandom()
        self._items = []
        self.load(items)
        
    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)
        
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
            
    def __call__(self):
        self.pick()

In [4]:
# lotto.py

import random
# from tombola import Tombola

class LotteryBlower(Tombola):
    def __init__(self, iterable):
        self._balls = list(iterable) # 사본으로 저장
        
    def load(self, iterable):
        self._balls.extend(iterable)
        
    def pick(self):
        try:
            pos = random.randrange(len(self._balls))
        except ValueError:
            raise LookupError('pick from empty BingCage')
        return self._balls.pop(pos)
    
    def loaded(self):
        return bool(self._balls)
    
    def inspect(self):
        return tuple(sorted(self._balls)) #매서드 오바라이드

### 11.7.3 가상 서브클래스
- 실제로 상속하지 않더라도 가상의 서브클래스로 등록 가능 
- 해당클래스가 ABC에 정의된 인터페이스를 충실히 구현하겠다는 약속
- 충실히 구현되지 않을 경우 런타임 예외 발생
- register() 매서드를 호출시, 클래스 등록 
- issubclass(), isinstance() 함수에 의해서는 인식이 되지만 상속된 메서드나 속성은 없음

In [7]:
# 예제 11-14 
# Tombola의 가상 서브클래스 TomboList 
from random import randrange
# from tombola import Tombola

@Tombola.register #가상서브클래스 등록 
class TomboList(list): #실제로는 list를 상속
    def pick(self):
        if self: # list 에서 __bool__을 상속받음, empty list 가 아닐 경우 True
            position = randrange(len(self))
            return self.pop(position)
        else:
            raise LookupError('pop from empty TomboList')

load = list.extend #TomboList.load 에 list.extend 할당

def loaded(self):
    return bool(self)

def inspect(self):
    return tuple(sorted(self))

print(issubclass(TomboList, Tombola))
print(isinstance(TomboList(range(100)), Tombola))

True
True


- 상속은 Method Resolution Order 를 담은 __mro__ 특별 클래스 속성에 의해 운영됨 
- 파이썬이 메서드를 검색할 순서대로 자신과 자신의 슈퍼클래스를 나열 

In [8]:
TomboList.__mro__ # 진짜 슈퍼클래스인 list, object 만 있음

(__main__.TomboList, list, object)